In [1]:
import threading
import random
import os
import xml.etree.ElementTree as ET


In [2]:
# Define the maximum size of the buffer
BUFFER_SIZE = 10
# Define the semaphore for accessing the buffer
mutex = threading.Semaphore(1)

# Define the semaphores for producer and consumer
full = threading.Semaphore(0)
empty = threading.Semaphore(BUFFER_SIZE)

# Define the buffer
buffer = []

In [3]:
# Define the ITstudent class
class ITstudent:
    def __init__(self):
        self.name = self.generate_name()
        self.id = self.generate_id()
        self.programme = self.generate_programme()
        self.courses = self.generate_courses()
        self.marks = self.generate_marks()
    
    def generate_name(self):
        names = ['Wandile','Sifiso','Ncamiso','Mpendulo','Sphamandla','Thando','Sebenele',
                 'Qiniso', 'Simphiwe','Magamuka']
        return random.choice(names)
    
    def generate_id(self):
        return str(random.randint(10000000, 99999999))
    
    def generate_programme(self):
        programme = 'Information Technology'
        return programme
    
    def generate_courses(self):
        courses = ['Database and Design', 'Computer Networking', 'Security', 
                   'Data Structures and Algorithms', 'Software Engineering',
                   'Modern Operating Systems', 'Web Technology and Development',
                   'Data Mining', 'Graphics']
        return random.sample(courses, random.randint(5,5))
                             
    def generate_marks(self):
        return {course: random.randint(0, 100) for course in self.courses}

In [4]:
# Define the producer function
def producer():
        global buffer
        for i in range(10):
            # Generate a new ITstudent object
            student = ITstudent()
        
            # Wrap student information into XML
            root = ET.Element("student")
            name = ET.SubElement(root, "name")
            name.text = student.name
            id = ET.SubElement(root, "id")
            id.text = student.id
            Programme = ET.SubElement(root, "programme")
            Programme.text = student.programme
            courses = ET.SubElement(root, "courses")
            for course, mark in student.marks.items():
                course_elem = ET.SubElement(courses, "course")
                course_elem.set("name", course)
                course_elem.set("mark", str(mark))
            filename = f"Student{i+1}.xml"
            tree = ET.ElementTree(root)
            tree.write(filename)
        
            # Acquire the empty semaphore
            empty.acquire()
            # Acquire the mutex semaphore
            mutex.acquire()
            # Add the filename to the buffer
            buffer.append(i+1)
            #print(f"Producer produced: {filename}")
            # Release the mutex semaphore
            mutex.release()
            # Release the full semaphore
            full.release()



In [5]:
# Define the consumer function
def consumer():
    global buffer
    while True:
        # Acquire the full semaphore
        full.acquire()
        # Acquire the mutex semaphore
        mutex.acquire()
        # Get the filename from the buffer
        filename = f"student{buffer.pop(0)}.xml"
        #print(f"Consumer consumed: {filename}")
        # Release the mutex semaphore
        mutex.release()
        # Release the empty semaphore
        empty.release()
        
        # unwrap student information from XML
        tree = ET.parse(filename)
        root = tree.getroot()
        student = ITstudent()
        student.name = root.find("name").text
        student.id = root.find("id").text
        student.programme = root.find("programme").text
        student.marks = {}
        for course_elem in root.find("courses").iter("course"):
            course = course_elem.get("name")
            mark = int(course_elem.get("mark"))
            student.marks[course] = mark
        # Calculate the average mark
        avg_mark = sum(student.marks.values()) / len(student.marks)
        # Determine the pass/fail status
        pass_fail = "Pass" if avg_mark >= 50 else "Fail"
        
        # Print the student information
        print(f"Name: {student.name}")
        print(f"ID: {student.id}")
        print(f"Programme: {student.programme}")
        print("Courses:")
        for course, mark in student.marks.items():
            print(f"- {course}: {mark}")
        print(f"Average mark: {avg_mark}")
        print(f"Pass/Fail: {pass_fail}")
        print("\n")
        # Clear the XML file
        os.remove(filename)

In [ ]:
# Create the producer and consumer threads
producer_thread = threading.Thread(target=producer)
consumer_thread = threading.Thread(target=consumer)

# Start the threads
producer_thread.start()
consumer_thread.start()

# Wait for the threads to finish
producer_thread.join()
consumer_thread.join()

Name: Sifiso
ID: 61856452
Programme: Information Technology
Courses:
- Computer Networking: 87
- Data Mining: 55
- Modern Operating Systems: 73
- Data Structures and Algorithms: 5
- Graphics: 12
Average mark: 46.4
Pass/Fail: Fail


Name: Ncamiso
ID: 40409504
Programme: Information Technology
Courses:
- Data Structures and Algorithms: 17
- Modern Operating Systems: 41
- Computer Networking: 63
- Graphics: 66
- Database and Design: 33
Average mark: 44.0
Pass/Fail: Fail


Name: Wandile
ID: 62205343
Programme: Information Technology
Courses:
- Modern Operating Systems: 25
- Data Structures and Algorithms: 13
- Database and Design: 5
- Computer Networking: 3
- Data Mining: 91
Average mark: 27.4
Pass/Fail: Fail


Name: Simphiwe
ID: 60491674
Programme: Information Technology
Courses:
- Database and Design: 27
- Security: 66
- Data Mining: 25
- Data Structures and Algorithms: 68
- Modern Operating Systems: 10
Average mark: 39.2
Pass/Fail: Fail


Name: Qiniso
ID: 43854094
Programme: Information